In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch.nn.functional as F
from transformers import pipeline
import numpy as np


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# Load Aspect-Based Sentiment Analysis model
absa_tokenizer = AutoTokenizer.from_pretrained("yangheng/deberta-v3-base-absa-v1.1")
absa_model = AutoModelForSequenceClassification.from_pretrained("yangheng/deberta-v3-base-absa-v1.1")

# Load a traditional Sentiment Analysis model
sentiment_model_path = "moarslan/bert-base-turkish-sentiment-analysis"
sentiment_model = pipeline("sentiment-analysis", model=sentiment_model_path,
                          tokenizer=sentiment_model_path)

model_id = "moarslan/bert-base-turkish-cased-ner"
ner = pipeline("ner",model = model_id)


In [ ]:
sentence = """Spotify'da müzik dinlerken hiçbir problem yaşıyorum, @SoundCloud üzerinden yayınları dinlemek bazen kesintiye uğruyor ve bu durum oldukça sinir bozucu."""
print(f"Sentence: {sentence}")

preds= ner(sentence, aggregation_strategy = "average") ## max average simple first seçenekleri var çıktıları değiştiriyor.

preds_df=pd.DataFrame(preds)
aspect = preds_df['word'].tolist()

# Sonuçları yazdır
print(aspect)

sentiment_label=['olumsuz', 'notr', 'olumlu']

for asp in aspect:
    inputs = absa_tokenizer(f"[CLS] {sentence} [SEP] {asp} [SEP]", return_tensors="pt")
    #print("inputs:", inputs)
    outputs = absa_model(**inputs)
    # print("output:", outputs.logits.shape)
    probs = F.softmax(outputs.logits, dim=1)
    probs = probs.detach().numpy()[0]
    predicted_class = np.argmax(probs)
    print("SONUC: ", sentiment_label[predicted_class])
    print("En yüksek olasılığa sahip sınıf:", predicted_class)
    for prob, label in zip(probs, ["olumsuz", "notr", "olumlu"]):
      print(f"Label {label}: {prob}")
